In [22]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [15]:
import pandas as pd
import pyreadr
import os
from newscatcherapi import NewsCatcherApiClient
import requests
from googleapiclient.discovery import build

# Get Wisconsin sources

In [5]:
circ_data = pyreadr.read_r("../aam_media_intelligence_center/circulation-zipcode/circulation data for elliott/aam_raw.RData")

In [6]:
circ_data = circ_data['aam_all']

In [7]:
wisconsin_newspaper = circ_data[circ_data.np_state == 'WI'].drop_duplicates(subset=['np_name']).np_name

In [8]:
already_available = pd.read_csv('../aam_media_intelligence_center/aam_mapping.csv')

In [30]:
wisconsin_newspaper = pd.merge(wisconsin_newspaper,already_available, on='np_name', how='left')

In [23]:
cse_id = os.getenv('GOOGLE_CX')

In [25]:
# get the websites of the newspapers via google search

def get_website(query):
    api_key = os.getenv('GOOGLE_API_KEY')
    cse_id = os.getenv('GOOGLE_CX')
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=query, cx=cse_id).execute()
    return res

In [26]:
res = get_website("THE POST-CRESCENT")            

In [28]:
def get_first_results_url(res):
    return res['items'][0]['link']

In [35]:
all_results = []
all_sites = []
for _,np in wisconsin_newspaper.iterrows():
    _res = get_website(np['np_name'])
    all_results.append(_res)
    all_sites.append(get_first_results_url(_res))
    



In [37]:
wisconsin_newspaper['website_raw'] = all_sites

In [43]:
wisconsin_newspaper["website_core"]=wisconsin_newspaper.website_raw.str.split('/').str[2].str.removeprefix("www.")

In [44]:
wisconsin_newspaper.to_csv('../data/wisconsin_newspaper.csv', index=False)

In [46]:
wisconsin_newspaper.drop_duplicates(subset=['website_core'])
# no longer available CUDAHY REMINDER ENTERPRISE, OAK CREEK PICTORIAL	


,np_name,np_name_adj,domain,website_raw,website_core
0,THE POST-CRESCENT,the post-crescent,NaN,https://www.postcrescent.com/,postcrescent.com
1,THE DAILY PRESS,the daily press,NaN,https://www.dailypress.com/,dailypress.com
2,BARABOO NEWS REPUBLIC,baraboo news republic,NaN,https://wiscnews.com/news/community/baraboonew...,wiscnews.com
3,CHIPPEWA HERALD,chippewa herald,NaN,https://chippewa.com/,chippewa.com
6,EAU CLAIRE LEADER-TELEGRAM,eau claire leader-telegram,NaN,https://www.leadertelegram.com/,leadertelegram.com
7,THE REPORTER,the reporter,NaN,https://www.thereporter.com/,thereporter.com
8,GREEN BAY PRESS GAZETTE,green bay press gazette,NaN,https://www.greenbaypressgazette.com/,greenbaypressgazette.com
9,THE JANESVILLE GAZETTE,the janesville gazette,NaN,https://www.gazettextra.com/,gazettextra.com
10,KENOSHA NEWS (MORNING & SUNDAY),kenosha news (morning & sunday),NaN,https://kenoshanews.com/,kenoshanews.com
11,THE LA CROSSE TRIBUNE,the la crosse tribune,NaN,https://lacrossetribune.com/,lacrossetribune.com


# Check available sources from wisconsin on newscatcher


In [10]:
api_key = os.getenv("NEWS_CATCHER_API_KEY")

In [13]:
newscatcherapi = NewsCatcherApiClient(x_api_key=api_key)


In [47]:
sources=wisconsin_newspaper.website_core.dropna().tolist()

In [62]:
sources = [s for s in sources if s not in ['oakcreeklibrary.org','loc.gov']]


In [64]:
news_api_res = newscatcherapi.get_search_all_pages(q="*", page_size=1000, sources=sources)

1 page is going to be extracted
Total number of found articles => 6643.
Total number of pages 7.
2/7 page is going to be extracted
3/7 page is going to be extracted
4/7 page is going to be extracted
5/7 page is going to be extracted
6/7 page is going to be extracted
7/7 page is going to be extracted


In [65]:
news_api_res

{'status': 'ok',
 'total_hits': 6643,
 'page': 1,
 'total_pages': 7,
 'page_size': 1000,
 'articles': [{'title': 'Menendez brothers murder case: LA district attorney reviewing new evidence',
   'author': 'David Matthews',
   'published_date': '2024-10-04 00:16:51',
   'published_date_precision': 'full',
   'link': 'https://www.nydailynews.com/2024/10/03/menendez-brothers-murders-new-evidence-los-angeles-district-attorney',
   'clean_url': 'nydailynews.com',
   'excerpt': 'Los Angeles County district attorney George Gascon said Thursday that his office had received new evidence in the infamous Menendez brothers murder case that may eventually lead to their release fr…',
   'summary': "Los Angeles County District Attorney George Gascon said Thursday his office had received new evidence in the infamous Menendez brothers murder case that may eventually lead to their release from prison.\nGascon said at a press conference the convictions of Erik and Lyle Menendez for killing their parents i

In [66]:
len(news_api_res["articles"])

6643

In [67]:
returned_sources = [x['clean_url'] for x in news_api_res["articles"]]


In [69]:
list(set(returned_sources))

['newmedia-wi.com',
 'madison.com',
 'sheboyganpress.com',
 'wiscnews.com',
 'dailypress.com',
 'news-graphic.com',
 'jsonline.com',
 'wausaudailyherald.com',
 'dailytribune.com',
 'gmtoday.com',
 'nydailynews.com',
 'chippewa.com',
 'greenbaypressgazette.com',
 'thenorthwestern.com',
 'captimes.com',
 'lacrossetribune.com',
 'htrnews.com',
 'postcrescent.com',
 'discoverhometown.com',
 'kenoshanews.com',
 'thereporter.com',
 'stevenspointjournal.com',
 'gazettextra.com',
 'journaltimes.com',
 'marshfieldnewsherald.com',
 'sfbayview.com',
 'tennessean.com',
 'leadertelegram.com']

In [70]:
wisconsin_newspaper['returned_sources'] = wisconsin_newspaper.website_core.isin(returned_sources)

In [72]:
wisconsin_newspaper.to_csv('../data/wisconsin_newspaper.csv', index=False)